# OLD VERSION: Sol Golomb’s Rectangle Puzzle

This problem was presented by Gary Antonik in his 14/4/14 New York Times [Numberplay column](http://wordplay.blogs.nytimes.com/2014/04/14/rectangle). 

>Say you’re given the following challenge: create a set of five rectangles that have sides of length 1, 2, 3, 4, 5, 6, 7, 8, 9 and 10 units. You can combine sides in a variety of ways: for example, you could create a set of rectangles with dimensions 1 x 3, 2 x 4, 5 x 7, 6 x 8 and 9 x 10.
>
>1. How many different sets of five rectangles are possible?
>
>2. What are the maximum and minimum values for the total areas of the five rectangles?
<p><b>Bonus Challenges</b><p>
>
>3. What other values for the total areas of the five rectangles are possible?
>
>4. Which sets of rectangles may be assembled to form a square?

To me, these are interesting questions because, first, I have personal connections to Solomon Golomb (my former colleague at USC) and to puzzle author Nelson Blachman (the father of my colleague Nancy Blachman)and to Antonik (a fun guy I've met with), and second, I find it interesting that the problems span the range from mathematical to computational. I wonder how the way I look at the problems compares to others who have a different mix of these two disciplines.

## 1. How many different sets of five rectangles are possible?

This is a basic [combinatorics](http://en.wikipedia.org/wiki/Combinatorics) or counting problem. I will present *three* methods to count the sets. If all goes right, they will give the same answer. The example set of rectangles given in the problem was

> {1 &times; 3, 2 &times; 4, 5 &times; 7, 6 &times; 8, 9 &times; 10}
    
and in general it would be

> {A &times; B, C &times; D, E &times; F, G &times; H, I &times; J}

The question is: how many distinct ways can we assign the numbers 1 through 10 to the variables A through J?
    
**Method 1: Count all permutations and divide by repetitions:** There are 10 variables to be filled, so there are 10! = 3,628,800 permutations.  But if we fill the first two variables with 1 &times; 3, that is the same rectangle as 3 &times; 1. So divide 10! by 2<sup>5</sup> to account for the fact that each of 5 rectangles can appear 2 ways.  Similarly, if we fill A and B with 1 &times; 3, that yields the same set as if we filled C and D with 1 &times; 3.  So divide again by 5! (the number of permutations of 5 things) to account for this.
That gives us <a href="https://www.google.com/search?ie=UTF-8#q=10!+%2F+2%5E5+/+5!">10! / 2<sup>5</sup> / 5!</a> = 945.  (It is always gratifying when this "count and divide" method comes out to a whole number.)

**Method 2: Count without repetitions**: The example set is presented in [lexicographical order](http://en.wikipedia.org/wiki/Lexicographical_order): in each rectangle the smaller component is listed first, and in the set, the rectangles with smaller first components are listed first.  An alternate to "count and divide" is to count directly how many sets there are in lexicographical order.  We'll work from left to right.  How many choices are there for variable A?  Only one: A must always be 1, because we agreed that the smallest number comes first. Then, given A, there are 9 remaining choices for B.  For C, given A and B, there is again only one choice: C must be the smallest of the remaining 8 numbers (it will be 3 if the first rectangle was 1 &times; 2; otherwise it will be 2, but either way there is only one choice).  That leaves 7 choices for D, 5 for F, 3 for H and 1 for J. And 9 &times; 7 &times; 5 &times; 3 &times; 1 = 945. (It is always gratifying when two methods give the same answer.)
          
**Method 3: Write a program to enumerate the sets:** We'll represent the 1 &times; 3 rectangle as the tuple `(1, 3)` and the example set of rectangles as the set

> {(1, 3), (2, 4), (5, 7), (6, 8), (9, 10)}

To generate all the sets of rectangles, we'll follow method 2: given a set of sides, first make all possible rectangles `(A, B)` where `A` is the shortest side, and `B` ranges over all the other sides.  For each of `(A, B)` rectangle, consider all possible `(C, D)` rectangles, and so on. This is easy to write out if we know there will be exactly 5 rectangles in a set:

In [1]:
def sets_of_rectangles(sides):
    "Given exactly 10 sides, yield all sets of rectangles that can be made from them."
    for (A, B) in each_rectangle(sides):
        for (C, D) in each_rectangle(sides - {A, B}):
            for (E, F) in each_rectangle(sides - {A, B, C, D}):
                for (G, H) in each_rectangle(sides - {A, B, C, D, E, F}):
                    for (I, J) in each_rectangle(sides - {A, B, C, D, E, F, G, H}):
                        yield {(A, B), (C, D), (E, F), (G, H), (I, J)}
                        
def each_rectangle(sides):
    "Yield all (A, B) pairs, where A is minimum of sides, and B ranges over all other sides."
    A = min(sides)
    for B in (sides - {A}):
        yield (A, B)    

In [2]:
def sets_of_rectangles(sides):
    "Given exactly 10 sides, yield all sets of rectangles that can be made from them."
    return [{(A, B), (C, D), (E, F), (G, H), (I, J)}
            for (A, B) in each_rectangle(sides)
            for (C, D) in each_rectangle(sides - {A, B})
            for (E, F) in each_rectangle(sides - {A, B, C, D})
            for (G, H) in each_rectangle(sides - {A, B, C, D, E, F})
            for (I, J) in each_rectangle(sides - {A, B, C, D, E, F, G, H})]
                        
def each_rectangle(sides):
    "Yield all (A, B) pairs, where A is minimum of sides, and B ranges over all other sides."
    A = min(sides)
    return [(A, B)
            for B in sides if B is not A]  

In [3]:
sides = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
sets = sets_of_rectangles(sides)
len(sets)

945

(Gratifying that once again we get the same answer.)  But I don't like the fact that we are limited to exactly 10 sides, and that some of the code is [repeated](http://en.wikipedia.org/wiki/Don't_repeat_yourself) five times.  So let's convert the nested-iterated algorithm into a *recursive* algorithm that chooses the first rectangle (in the same way as before), but then recursively solves for the remaining sides and unions the set of rectangles from the remaining sides with the first rectangle.  (Note we have to also correctly handle the case when there are no sides; then there is one way to make a set of rectangles: the empty set.)

In [4]:
def sets_of_rectangles(sides):
    "Given a set of sides, yield all sets of rectangles that can be made from sides."
    if sides:
        A = min(sides)
        return [{(A, B)}.union(rest)
                for B in (sides - {A})
                for rest in sets_of_rectangles(sides - {A, B})]
    else:
        return [set()]

In [5]:
# For example, with 6 sides:
sets_of_rectangles({1, 2, 3, 4, 5, 6})

[{(1, 2), (3, 4), (5, 6)},
 {(1, 2), (3, 5), (4, 6)},
 {(1, 2), (3, 6), (4, 5)},
 {(1, 3), (2, 4), (5, 6)},
 {(1, 3), (2, 5), (4, 6)},
 {(1, 3), (2, 6), (4, 5)},
 {(1, 4), (2, 3), (5, 6)},
 {(1, 4), (2, 5), (3, 6)},
 {(1, 4), (2, 6), (3, 5)},
 {(1, 5), (2, 3), (4, 6)},
 {(1, 5), (2, 4), (3, 6)},
 {(1, 5), (2, 6), (3, 4)},
 {(1, 6), (2, 3), (4, 5)},
 {(1, 6), (2, 4), (3, 5)},
 {(1, 6), (2, 5), (3, 4)}]

In [7]:
# Reaffirm the answer with 10 sides
len(sets_of_rectangles(sides))

945

I don't want to print all 945 sets, but let's peek at every 100th one:

In [8]:
sets_of_rectangles(sides)[::100]

[{(1, 2), (3, 4), (5, 8), (6, 9), (7, 10)},
 {(1, 2), (3, 10), (4, 6), (5, 9), (7, 8)},
 {(1, 3), (2, 10), (4, 9), (5, 7), (6, 8)},
 {(1, 4), (2, 10), (3, 8), (5, 9), (6, 7)},
 {(1, 5), (2, 9), (3, 6), (4, 10), (7, 8)},
 {(1, 6), (2, 9), (3, 10), (4, 7), (5, 8)},
 {(1, 7), (2, 9), (3, 8), (4, 10), (5, 6)},
 {(1, 8), (2, 7), (3, 5), (4, 10), (6, 9)},
 {(1, 9), (2, 7), (3, 10), (4, 6), (5, 8)},
 {(1, 10), (2, 7), (3, 8), (4, 9), (5, 6)}]

## 2. What are the maximum and minimum values for the total areas of the five rectangles?

I think I know this one, but I'm not completely sure.  I know that a rectangle with a fixed perimeter has maximum area when it is a square. My guess is that the maximum total area occurs when each rectangle is *almost* square: a 9 &times; 10 rectangle; 7 &times; 8; and so on.  So that would give us a maximum area of:

In [9]:
(1 * 2) + (3 * 4) + (5 * 6) + (7 * 8) + (9  * 10)

190

And the minimum area should be when the rectangles deviate the most from squares: a 1 &times; 10, and so on:

In [10]:
(1 * 10) + (2 * 9) + (3 * 8) + (4 * 7) + (5 * 6)

110

Since I am not sure, I will double check by computing the set of all possible areas and then asking for the min and max of the set:

In [15]:
def total_area(rectangles): 
    "The total area of a set of rectangles."
    return sum((w * h) for (w, h) in rectangles)

areas = [total_area(s) for s in sets]

In [16]:
max(areas)

190

In [17]:
min(areas)

110

This verifies that the maximum and minimum are what I thought they were. But I still don't think I completely understand the situation.  For example, suppose there are *N* sides that are not necessarily consecutive integers.  Will the maximum total area always be formed by combining the two biggest sides, and so on?

## 3. What other values for the total areas of the five rectangles are possible?

I have no idea how to figure this out mathematically from first principles, but it is easy to compute with the code we already have:

In [20]:
print(sorted(set(areas)))

[110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 177, 178, 179, 180, 181, 182, 183, 184, 186, 187, 190]


Is there a more succint way to describe these values?

In [21]:
set(range(110, 191)) - set(areas)

{176, 185, 188, 189}

Yes: All the integers between 110 and 190 inclusive, except 176, 185, 188, and 189.

## 4. Which sets of rectangles may be assembled to form a square?

The only way I can think about this is to write a program; I don't see any way to work it out by hand.  I do know that the total area will have to be a perfect square, and that in the range of areas (110 to 190) the only perfect squares are 11<sup>2</sup> = 121, 12<sup>2</sup> = 144, and 13<sup>2</sup> = 169. We can find the rectangle sets that have a perfect square as their total area:

In [22]:
[s for s in sets if total_area(s) in {121, 144, 169}]

[{(1, 2), (3, 5), (4, 9), (6, 10), (7, 8)},
 {(1, 2), (3, 7), (4, 6), (5, 10), (8, 9)},
 {(1, 2), (3, 7), (4, 9), (5, 6), (8, 10)},
 {(1, 2), (3, 8), (4, 5), (6, 10), (7, 9)},
 {(1, 3), (2, 5), (4, 8), (6, 9), (7, 10)},
 {(1, 3), (2, 7), (4, 5), (6, 10), (8, 9)},
 {(1, 3), (2, 7), (4, 8), (5, 6), (9, 10)},
 {(1, 3), (2, 9), (4, 10), (5, 7), (6, 8)},
 {(1, 3), (2, 10), (4, 7), (5, 9), (6, 8)},
 {(1, 3), (2, 10), (4, 8), (5, 7), (6, 9)},
 {(1, 4), (2, 5), (3, 7), (6, 9), (8, 10)},
 {(1, 5), (2, 3), (4, 9), (6, 7), (8, 10)},
 {(1, 5), (2, 4), (3, 8), (6, 7), (9, 10)},
 {(1, 5), (2, 6), (3, 8), (4, 10), (7, 9)},
 {(1, 5), (2, 7), (3, 4), (6, 8), (9, 10)},
 {(1, 6), (2, 3), (4, 8), (5, 7), (9, 10)},
 {(1, 6), (2, 9), (3, 10), (4, 8), (5, 7)},
 {(1, 6), (2, 10), (3, 8), (4, 9), (5, 7)},
 {(1, 6), (2, 10), (3, 9), (4, 7), (5, 8)},
 {(1, 7), (2, 4), (3, 8), (5, 9), (6, 10)},
 {(1, 7), (2, 9), (3, 5), (4, 6), (8, 10)},
 {(1, 7), (2, 10), (3, 6), (4, 9), (5, 8)},
 {(1, 8), (2, 6), (3, 10), (4, 9

In [23]:
len(_)

35

So 35 out of 945 rectangle sets *might* be assembled into a square; we don't know yet. I would like to see *how* the rectangles are packed into the square, not just *which* sets can be.  To start, I'll represent a *Board* as a two-dimensional array: a list of rows, each of which is a list of cells, with the idea that each cell will be covered by a rectangle.  We'll initialize each cell to be `empty`:

In [24]:
empty = (0, 0)

def Board(width, height):
    "Make an empty board of size width by height."
    return [[empty for x in range(width)] for y in range(height)]

def Square(size): return Board(size, size)    

In [25]:
Square(5)

[[(0, 0), (0, 0), (0, 0), (0, 0), (0, 0)],
 [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0)],
 [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0)],
 [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0)],
 [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]]

Now we need to be able to place rectangles onto a board.  The function `place_rectangle_at` places a single rectangle of width *w* and height *h* onto a board at position (x<sub>0</sub>, y<sub>0</sub>).  If the rectangle overlaps a non-empty cell, or goes off the board, we return `None` to indicate that this is not a legal packing. If the rectangle does fit, we return a new board with any old rectangles plus the new one (note we do not modify the old board):

In [28]:
def place_rectangle_at(rect, board, pos):
    """Place the rectangle of size (w, h) onto board at position (x0, y0).
    Return a new board, or None if the rectangle cannot be placed."""
    (w, h) = rect
    (x0, y0) = pos
    newboard = [list(row) for row in board] # copy old board
    try:
        for x in range(x0, x0+w):
            for y in range(y0, y0+h):
                if newboard[y][x] is not empty:
                    return None   
                newboard[y][x] = rect
        return newboard
    except IndexError: # went off the board
        return None

Now we place a 3 &times; 4 rectangle in the upper left corner of the 5 &times; 5 square:

In [29]:
place_rectangle_at((3, 4), Square(5), (0, 0))

[[(3, 4), (3, 4), (3, 4), (0, 0), (0, 0)],
 [(3, 4), (3, 4), (3, 4), (0, 0), (0, 0)],
 [(3, 4), (3, 4), (3, 4), (0, 0), (0, 0)],
 [(3, 4), (3, 4), (3, 4), (0, 0), (0, 0)],
 [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]]

That looks right, but it is a bit difficult to read. Let's figure out how to display a board with pretty multi-colored rectangles.  I will use some  APIs that are specific to IPython, and some HTML and CSS, so feel free to skip these details if you want; all you have to know is that `display_board(board)` causes the board to be displayed.

(If you *are* interested in the details, note that we are creating an HTML table, where each cell (each `<td>` entry) has its own color.  We use the color `#f0f0f0`, a light grey, for empty, and for non-empty rectangles, we make the color less red the wider it is, and less green the taller it is (which has the effect that big rectangles are mostly blue). In Python `0xf0` is the hex notation for the integer `240`.)

In [32]:
from IPython.display import HTML, display

def display_board(board):
    "Display a representation of this board."
    display('Not a legal board' if not board else
            HTML('<table style="border: solid black 4px;">' +
                 cat(html_row(r) for r in board) +
                 '</table>'))
        
def color(w, h):
    return "#{:2x}{:2x}{:2x}".format(0xf0-w*22, 0xf0-h*22, 0xf0)
        
def html_row(row): 
    return '\n<tr>' + cat(html_cell(rect) for rect in row)
        
def html_cell(rect): 
    (w, h) = rect
    return ('<td style="text-align:center; width:1.3em; height:1.3em;' +
            'background-color:{}">{}{}</td>'.format(color(w, h), w%10, h%10))

cat = ''.join

In [31]:
display_board(place_rectangle_at((3, 4), Square(5), (0, 0)))

34,34,34,00,00
34,34,34,00,00
34,34,34,00,00
34,34,34,00,00
00,00,00,00,00


Much prettier! Here is how to place more than rectangle:

In [ ]:
board1 = Square(10)
board2 = place_rectangle_at((5, 7), board1, (0, 0))
board3 = place_rectangle_at((4, 9), board2, (5, 1))
display_board(board3)

In [ ]:
# Now try to place a third rectangle that does not fit; should fail
display_board(place_rectangle_at((8, 2), board3, (0, 5)))

Now we need a strategy for packing a set of rectangles onto a board.  I know that many variants of [bin packing problems](http://en.wikipedia.org/wiki/Bin_packing_problem) are NP-hard, but we only have 5 rectangles, so it should be easy: just exhaustively try each rectangle in each possible position in both possible orientations (horizontal and vertical). We can do this two ways:

> Way 1: Considering the rectangles in a fixed order, try every possible position for each rectangle.

or

> Way 2: Considering the positions in a fixed order, try every possible rectangle for each position.

In Way 1, we could pre-sort the rectangles (say, biggest first).  Then we try to put the biggest rectangle in all possible positions on the board, and for each position that fits, try putting the second biggest rectangle in all remaining positions, and so on.  

In Way 2, we consider the positions in some fixed order; say top-to-bottom, left-to right.  Take the first empty position (the upper left corner).  Try putting each of the rectangles there, and for each one that fits, try the next empty position, and consider all possible rectangles to go there, and so on.

Which approach is better?  I'm not sure; let's compare. As a wild guess, assume there are on average about 10 ways to place a rectangle on a board.  Then Way 1 will look at 10<sup>5</sup> = 100,000 combinations.  For Way 2, there are only 5! = 120 permutations of rectangles, and each rectangle can go either horizontaly or verticaly, so that's 5! &times; 2<sup>5</sup> = 3840.  Since 3840 &lt; 100,000, I'll go with Way 2.  Here is a more precise description:

> Way 2: To **pack** a set of rectangles onto a board, find the first empty cell on the board.  Try in turn to place each rectangle (in either orientation) at that position. For each one that fits, try to recursively **pack** the remaining rectangles, and return the resulting board if one of these recursive calls succeeds.  If none succeeds, return None.  *Details:* If there are no rectangles to pack, the solution is the original board.  If the board is illegal, we can't place anything on it, so return None. The order we choose for the first empty cell is arbitrary; we will go top-to-bottom, left-to-right.

In [ ]:
def pack(rectangles, board):
    """Find a way to pack all rectangles onto board and return the packed board,
    or return None if not possible."""
    if not rectangles:
        return board # Placing no rectangles give you the original board
    elif not board:
        return None  # Can't put rectangles on an illegal board
    else:
        pos = first_empty_cell(board)
        for (w, h) in rectangles: # for each rectangle
            for rect in [(w, h), (h, w)]: # in each orientation
                board2 = place_rectangle_at(rect, board, pos)
                solution = pack(rectangles - {(w, h)}, board2)
                if solution:
                    return solution
        return None
    
def first_empty_cell(board):
    "The uppermost, leftmost empty cell position on board."
    for (y, row) in enumerate(board):
        for (x, cell) in enumerate(row):
            if cell is empty:
                return (x, y)

Let's give it a try:

In [ ]:
display_board(pack({(5, 7), (4, 9)}, Square(10)))

Looks good! We see that the first rectangle was placed in the upper left, then the second was placed at the first empty cell after the first rectangle.  

Now we will define the function `display_packable` which takes a collection of sets of rectangles and displays the ones that can be packed into a square.  Most of the work is done by `pack`:

In [ ]:
perfect = {i**2:i for i in range(100)}

def packable(sets):
    "Go through all sets of rectangles and display those that can be packed into a square."
    for rectangles in sets:
        A = total_area(rectangles)
        if A in perfect:
            board = pack(rectangles, Square(perfect[A]))
            if board:
                yield board

Finally, we can answer question 4:

In [ ]:
map(display_board, packable(sets))

So we see that out of the 945 sets of rectangles, and the 35 whose total area is a perfect square, there are only **four** that are packable. (I had a couple of ideas for how to make this faster, but I won't bother because it is plenty fast enough&mdash;that took less than a second.)